### VarOr/VarAnd
VarOr和VarAnd是演化算法中的两种范式。VarOr表示交叉和变异必须选择其中一种执行。VarAnd则相对自由，可以同时执行交叉和变异，也可以同时不执行它们。GP的原始论文使用的是VarOr。

In [1]:
import time

import numpy as np
from deap import base, creator, tools, gp


# 符号回归
def evalSymbReg(individual, pset):
    # 编译GP树为函数
    func = gp.compile(expr=individual, pset=pset)

    # 使用numpy创建一个向量
    x = np.linspace(-10, 10, 100)

    # 评估生成的函数并计算MSE
    mse = np.mean((func(x) - x ** 2) ** 2)

    return (mse,)


# 创建个体和适应度函数
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

In [2]:
import random

# 定义函数集合和终端集合
pset = gp.PrimitiveSet("MAIN", arity=1)
pset.addPrimitive(np.add, 2)
pset.addPrimitive(np.subtract, 2)
pset.addPrimitive(np.multiply, 2)
pset.addPrimitive(np.negative, 1)
pset.addEphemeralConstant("rand101", lambda: random.randint(-1, 1))
pset.renameArguments(ARG0='x')

# 定义遗传编程操作
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("evaluate", evalSymbReg, pset=pset)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr, pset=pset)

/home/glk/project/anaconda3/lib/python3.11/site-packages/deap/gp.py:257: RuntimeWarning: Ephemeral rand101 function cannot be pickled because its generating function is a lambda function. Use functools.partial instead.
  warnings.warn("Ephemeral {name} function cannot be "


DEAP默认使用VarAnd范式，如果我们想要实现VarOr，就需要自己修改eaSimple函数。当然，具体选择VarAnd还是VarOr要根据具体问题而定。目前尚无统一的结论表明哪种方式一定更好，需要根据问题的特性来决定。

In [3]:
from deap.algorithms import varOr
import numpy
from deap import algorithms


def eaSimple(population, toolbox, cxpb, mutpb, ngen, stats=None,
             halloffame=None, verbose=__debug__):
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    if halloffame is not None:
        halloffame.update(population)

    record = stats.compile(population) if stats else {}
    logbook.record(gen=0, nevals=len(invalid_ind), **record)
    if verbose:
        print(logbook.stream)

    # Begin the generational process
    for gen in range(1, ngen + 1):
        # Select the next generation individuals
        offspring = toolbox.select(population, len(population))

        # Vary the pool of individuals
        offspring = varOr(offspring, toolbox, len(offspring),cxpb, mutpb)

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Update the hall of fame with the generated individuals
        if halloffame is not None:
            halloffame.update(offspring)

        # Replace the current population by the offspring
        population[:] = offspring

        # Append the current generation statistics to the logbook
        record = stats.compile(population) if stats else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        if verbose:
            print(logbook.stream)

    return population, logbook


# 定义统计指标
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", numpy.mean)
mstats.register("std", numpy.std)
mstats.register("min", numpy.min)
mstats.register("max", numpy.max)

# 使用默认算法
start = time.time()
population = toolbox.population(n=300)
hof = tools.HallOfFame(1)
pop, log = algorithms.eaSimple(population=population,
                               toolbox=toolbox, cxpb=0.5, mutpb=0.2, ngen=50, stats=mstats, halloffame=hof,
                               verbose=True)
end = time.time()
print('time:', end - start)
print(str(hof[0]))

   	      	                    fitness                    	                      size                     
   	      	-----------------------------------------------	-----------------------------------------------
gen	nevals	avg    	gen	max   	min	nevals	std    	avg    	gen	max	min	nevals	std    
0  	300   	2676.63	0  	151631	0  	300   	8715.79	4.09667	0  	7  	2  	300   	1.66553
1  	181   	41585.6	1  	1.18774e+07	0  	181   	684483 	4.30333	1  	13 	2  	181   	1.97433
2  	188   	4889.34	2  	608604     	0  	188   	37082.5	4.39667	2  	15 	2  	188   	2.11013
3  	177   	4230.57	3  	153712     	0  	177   	19756.6	4.58667	3  	14 	2  	177   	2.11246
4  	196   	5564.12	4  	608604     	0  	196   	39150.9	4.67667	4  	20 	2  	196   	2.48169
5  	185   	46180.3	5  	1.18774e+07	0  	185   	685499 	4.57333	5  	20 	2  	185   	2.66795
6  	172   	43943.6	6  	1.23323e+07	0  	172   	710866 	4.41667	6  	16 	2  	172   	2.46097
7  	168   	4588.77	7  	159956     	0  	168   	24847.3	3.98333	7  	13 	2  	168   	1.7

## VarOr vs VarAnd

varAnd的工作原理：
```python
def varAnd(population, toolbox, cxpb, mutpb):
    offspring = list(population)  # 创建种群副本
    
    # 对每对个体应用交叉
    for i in range(1, len(offspring), 2):
        if random.random() < cxpb:  # 以cxpb的概率进行交叉
            offspring[i-1], offspring[i] = toolbox.mate(offspring[i-1], offspring[i])
            del offspring[i-1].fitness.values  # 删除适应度值
            del offspring[i].fitness.values
    
    # 对每个个体应用变异
    for i in range(len(offspring)):
        if random.random() < mutpb:  # 以mutpb的概率进行变异
            offspring[i], = toolbox.mutate(offspring[i])
            del offspring[i].fitness.values
            
    return offspring
```

varOr的工作原理：
```python
def varOr(population, toolbox, lambda_, cxpb, mutpb):
    offspring = []
    
    for _ in range(lambda_):  # 生成指定数量的后代
        op_choice = random.random()
        if op_choice < cxpb:  # 交叉
            ind1, ind2 = random.sample(population, 2)
            ind1, ind2 = toolbox.mate(ind1, ind2)
            del ind1.fitness.values
            offspring.append(ind1)
        elif op_choice < cxpb + mutpb:  # 变异
            ind = random.choice(population)
            ind, = toolbox.mutate(ind)
            del ind.fitness.values
            offspring.append(ind)
        else:  # 复制
            offspring.append(random.choice(population))
            
    return offspring
```

主要区别：

1. 操作方式：
   - varAnd：先进行交叉操作，然后进行变异操作，两个操作是独立的
   - varOr：在每次生成新个体时，只选择一种操作（交叉、变异或复制）

2. 概率含义：
   - varAnd：cxpb和mutpb分别是交叉和变异的独立概率
   - varOr：cxpb和mutpb的和必须小于1，剩余的概率用于直接复制

3. 种群大小：
   - varAnd：保持种群大小不变
   - varOr：可以通过lambda_参数控制产生的后代数量

选择建议：

1. 使用varAnd的情况：
   - 当你想要保持种群规模稳定
   - 当你希望交叉和变异操作能够独立发生
   - 在传统的遗传算法框架中（如eaSimple）

2. 使用varOr的情况：
   - 当你需要更灵活的种群大小控制
   - 在进化策略（ES）类型的算法中
   - 需要明确控制复制操作的概率时

在代码中使用了varOr，这是因为：
1. 它允许更灵活的种群管理
2. 每代都产生固定数量的新个体
3. 为种群引入了更多的多样性（通过复制操作）

观察代码中的参数：
```python
algorithms.eaSimple(population=population,
                   toolbox=toolbox, 
                   cxpb=0.5,  # 50%概率进行交叉
                   mutpb=0.2,  # 20%概率进行变异
                   # 剩余30%概率进行复制
                   ngen=50, 
                   stats=mstats, 
                   halloffame=hof,
                   verbose=True)
```

这些参数表明算法在保持一定探索性（通过交叉和变异）的同时，也通过复制操作保持了一定的稳定性。这种平衡对于大多数优化问题来说是合适的。